In [ ]:
!pip install wandb

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import wandb
from wandb.keras import WandbCallback

import numpy as np
import cv2

In [ ]:
wandb.login()

In [11]:
wandb.init(project="Sheykh Recognition", entity="gharabadiyan")

In [12]:
train_data_path = '/content/drive/MyDrive/Sheykh-Recognition'
width = height = 224
batch_size = 32

idg = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    brightness_range = (0.6,1.2),
    zoom_range = 0.1,
    shear_range = 0.3,
    rotation_range = 15,
    validation_split = 0.2
)

train_data = idg.flow_from_directory(
    train_data_path,
    target_size = (width,height),
    class_mode = "categorical",
    batch_size = batch_size,
    subset = "training"
)
val_data = idg.flow_from_directory(
    train_data_path,
    target_size = (width,height),
    class_mode = "categorical",
    batch_size = batch_size,
    subset = "validation"
)



Found 374 images belonging to 2 classes.
Found 92 images belonging to 2 classes.


## Model

In [13]:
base_model = tf.keras.applications.ResNet50V2(
    include_top=False,
    weights = 'imagenet',
    input_shape=(width,height,3),
    pooling = 'avg',
)

model = tf.keras.models.Sequential([
        base_model,
        Flatten(),
        Dense(1024,activation='relu'),
        Dense(256,activation='relu'),
        Dense(2,activation='softmax')

])

In [14]:
for layer in base_model.layers[:-4]:  #freeze layer
  layer.trainable = False

In [15]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [16]:
model.fit(train_data,validation_data=val_data,epochs=10,callbacks=[WandbCallback()])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Epoch 1/10
12/12 [==============================] - 68s 5s/step - loss: 0.7043 - accuracy: 0.7005 - val_loss: 0.2844 - val_accuracy: 0.8913 - _timestamp: 1657799655.0000 - _runtime: 90.0000
Epoch 2/10
12/12 [==============================] - 63s 5s/step - loss: 0.2296 - accuracy: 0.9118 - val_loss: 0.1893 - val_accuracy: 0.9239 - _timestamp: 1657799735.0000 - _runtime: 170.0000
Epoch 3/10
12/12 [==============================] - 64s 5s/step - loss: 0.1212 - accuracy: 0.9519 - val_loss: 0.1598 - val_accuracy: 0.9239 - _timestamp: 1657799817.0000 - _runtime: 252.0000
Epoch 4/10
12/12 [==============================] - 63s 5s/step - loss: 0.0479 - accuracy: 0.9840 - val_loss: 0.2908 - val_accuracy: 0.8913 - _timestamp: 1657799899.0000 - _runtime: 334.0000
Epoch 5/10
12/12 [==============================] - 63s 5s/step - loss: 0.0622 - accuracy: 0.9706 - val_loss: 0.2807 - val_accuracy: 0.8913 - _times

In [ ]:
model.save('Sheykh_Recognition.h5')